In [41]:
import mindspore as ms
import mindquantum as mq
from mindquantum.simulator import Simulator
from mindquantum.core.circuit import Circuit
from mindquantum.core.gates import X, RY, CNOT, Measure
from mindquantum.core.parameterresolver import ParameterResolver
import numpy as np
import scipy as scp

print('mindspore:', ms.__version__)
print('mindquantum:', mq.__version__)
print('numpy:', np.__version__)
print('scipy:', scp.__version__)

try:
    import openfermion as opf
    import openfermionpyscf as opf_scf
    print('openfermion:', opf.__version__)
    print('openfermionpyscf:', opf_scf.__version__)

    HAS_OPF = True
except:
    HAS_OPF = False

mindspore: 1.8.0
mindquantum: 0.8.0
numpy: 1.19.5
scipy: 1.7.3
openfermion: 1.4.0
openfermionpyscf: 0.5


In [42]:
if HAS_OPF:
    from mindquantum import Hamiltonian, get_qubit_hamiltonian
    from openfermion.chem import MolecularData
    from openfermionpyscf import run_pyscf

    mol = MolecularData([("H", (0, 0, 0)), ("H", (0, 0, 1))], "sto3g", multiplicity=1)
    mol = run_pyscf(mol, run_fci=1)
    ham = Hamiltonian(get_qubit_hamiltonian(mol))
    print('ref value:', mol.fci_energy)   # -1.1011503
else:
    from mindquantum.core.operators import QubitOperator

    paulis = [
      (-0.3276, ""),
      (0.1372, "Z0"),
      (0.1372, "Z1"),
      (-0.1304, "Z2"),
      (-0.1304, "Z3"),
      (0.1566, "Z0 Z1"),
      (0.1062, "Z0 Z2"),
      (0.1554, "Z0 Z3"),
      (0.1554, "Z1 Z2"),
      (0.1062, "Z1 Z3"),
      (0.1633, "Z2 Z3"),
      (0.0492, "X0 Y1 Y2 X3"),
      (-0.0492, "Y0 Y1 X2 X3"),
      (-0.0492, "X0 X1 Y2 Y3"),
      (0.0492, "Y0 X1 X2 Y3"),
    ]
    ham = Hamiltonian(sum([QubitOperator(string, coeff) for coeff, string in paulis]))
    print('ref value:', -1.1011503)

print('ham:')
ham

ref value: -1.1011503302326187
ham:


QubitOperator(({}, const: (-0.327608,0) []
{}, const: (0.137166,0) [Z0]
{}, const: (0.137166,0) [Z1]
{}, const: (-0.130363,0) [Z2]
{}, const: (-0.130363,0) [Z3]
{}, const: (0.156601,0) [Z0 Z1]
{}, const: (0.0491976,0) [Y0 X1 X2 Y3]
{}, const: (-0.0491976,0) [Y0 Y1 X2 X3]
{}, const: (-0.0491976,0) [X0 X1 Y2 Y3]
{}, const: (0.0491976,0) [X0 Y1 Y2 X3]
{}, const: (0.106229,0) [Z0 Z2]
{}, const: (0.155427,0) [Z0 Z3]
{}, const: (0.155427,0) [Z1 Z2]
{}, const: (0.106229,0) [Z1 Z3]
{}, const: (0.163268,0) [Z2 Z3]))

In [43]:
nq = 4
p = 0
param = lambda p: f'tht_{p}'

cir = Circuit()
for i in range(nq // 2):
    cir += X.on(i)
for i in range(nq):
    cir += RY(param(p)).on(i) ; p += 1
for i in range(1, nq):
    cir += CNOT.on(i, i - 1)
for i in range(nq):
    cir += RY(param(p)).on(i) ; p += 1

cir.summary()
print(cir)
cir.svg()

=================================Circuit Summary=================================
|Total number of gates  : 13.                                                   |
|Parameter gates        : 8.                                                    |
|with 8 parameters are  :                                                       |
|tht_0, tht_1, tht_2, tht_3, tht_4, tht_5, tht_6, tht_7                        .|
|Number qubit of circuit: 4                                                     |
q0: ──────X────────RY(tht_0)────●────RY(tht_4)────────────────────────────
                                │
q1: ──────X────────RY(tht_1)────X────────●────────RY(tht_5)───────────────
                                         │
q2: ──RY(tht_2)──────────────────────────X────────────●────────RY(tht_6)──
                                                      │
q3: ──RY(tht_3)───────────────────────────────────────X────────RY(tht_7)──


In [67]:
sim = Simulator('mqvector', cir.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham, cir)

sim

mqvector simulator with 4 qubits (little endian).
Current quantum state:
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

In [94]:
from mindquantum.framework import MQAnsatzOnlyLayer
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")

net = MQAnsatzOnlyLayer(grad_ops, weight='Zeros')
print(net)
print(net.trainable_params())
optimizer = ms.nn.Adam(net.trainable_params(), learning_rate=0.05)
optimizer_step = ms.nn.TrainOneStepCell(net, optimizer)

for iter in range (1000):
    e_i = optimizer_step()
    if iter % 100 == 0:
        print("Step %3d energy %.16f" % (iter, float(e_i)))

print("Optimization completed at step %3d" % (iter - 1))
print("Optimized energy: %20.16f" % e_i)
print("Optimized amplitudes: \n", net.weight.asnumpy())

MQAnsatzOnlyLayer<
  (evolution): MQAnsatzOnlyOps<4 qubits mqvector VQA Operator>
  >
[Parameter (name=ansatz_weight, shape=(8,), dtype=Float32, requires_grad=True)]
Step   0 energy -0.5816669464111328
Step 100 energy -0.5816669464111328
Step 200 energy -0.5816669464111328
Step 300 energy -0.5816669464111328
Step 400 energy -0.5816669464111328
Step 500 energy -0.5816669464111328
Step 600 energy -0.5816669464111328
Step 700 energy -0.5816669464111328
Step 800 energy -0.5816669464111328
Step 900 energy -0.5816669464111328
Optimization completed at step 998
Optimized energy:  -0.5816669464111328
Optimized amplitudes: 
 [0. 0. 0. 0. 0. 0. 0. 0.]


In [96]:
from scipy.optimize import minimize

theta = np.random.uniform(low=-1, high=1, size=[2*nq]) * 1e-3

def loss_fn(x):
    f, g = grad_ops(x)
    f, g = [x.squeeze() for x in [f, g]]
    return f.real, g.real

res = minimize(loss_fn, theta, method='BFGS', jac=True, tol=1e-6)
print('E0 energy:', res.fun)
print('params:', res.x)

E0 energy: -1.1011503302323775
params: [-3.52437151e-01  4.09113900e-07  1.01859074e-07 -5.92045284e-07
  5.59020644e-07 -3.14159326e+00 -3.36775470e-07  6.45338619e-07]
